In [1]:
import cv2
import numpy as np

import matplotlib.pyplot as plt

In [2]:
import dlib
dlib.__version__

'19.9.0'

In [3]:
import numpy
numpy.__version__

'1.11.3'

In [4]:
import pandas
pandas.__version__

'0.23.4'

In [8]:
path = r"F:\PYcode\coding\faceinfo\007.jpg"
img = cv2.imread(path)